In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing
import xgboost as xgb
import datetime

#load files
train = pd.read_csv('/Users/xiechenwei/Downloads/train.csv', parse_dates=['timestamp'])
test = pd.read_csv('/Users/xiechenwei/Downloads/test.csv', parse_dates=['timestamp'])
macro = pd.read_csv('/Users/xiechenwei/Downloads/macro.csv', parse_dates=['timestamp'])
id_test = test.id



/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
test.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,30474,2015-07-01,39.0,20.7,2,9,1,1998.0,1,8.9,...,8,0,0,0,1,10,1,0,14,1
1,30475,2015-07-01,79.2,NaN,8,17,1,0.0,3,1.0,...,4,1,1,0,2,11,0,1,12,1
2,30476,2015-07-01,40.5,25.1,3,5,2,1960.0,2,4.8,...,42,11,4,0,10,21,0,10,71,11
3,30477,2015-07-01,62.8,36.0,17,17,1,2016.0,2,62.8,...,1,1,2,0,0,10,0,0,2,0
4,30478,2015-07-01,40.0,40.0,17,17,1,0.0,1,1.0,...,5,1,1,0,2,12,0,1,11,1


In [3]:
train.head(n = 1)
print(train.shape)
print(macro.shape)

(30471, 292)
(2484, 100)


In [4]:
#multiplier = 0.969

#clean data
bad_index = train[train.life_sq > train.full_sq].index
train.ix[bad_index, "life_sq"] = np.NaN
equal_index = [601,1896,2791]
test.ix[equal_index, "life_sq"] = test.ix[equal_index, "full_sq"]
bad_index = test[test.life_sq > test.full_sq].index
test.ix[bad_index, "life_sq"] = np.NaN
bad_index = train[train.life_sq < 5].index
train.ix[bad_index, "life_sq"] = np.NaN
bad_index = test[test.life_sq < 5].index
test.ix[bad_index, "life_sq"] = np.NaN
bad_index = train[train.full_sq < 5].index
train.ix[bad_index, "full_sq"] = np.NaN
bad_index = test[test.full_sq < 5].index
test.ix[bad_index, "full_sq"] = np.NaN
kitch_is_build_year = [13117]
train.ix[kitch_is_build_year, "build_year"] = train.ix[kitch_is_build_year, "kitch_sq"]
bad_index = train[train.kitch_sq >= train.life_sq].index
train.ix[bad_index, "kitch_sq"] = np.NaN
bad_index = test[test.kitch_sq >= test.life_sq].index
test.ix[bad_index, "kitch_sq"] = np.NaN
bad_index = train[(train.kitch_sq == 0).values + (train.kitch_sq == 1).values].index
train.ix[bad_index, "kitch_sq"] = np.NaN
bad_index = test[(test.kitch_sq == 0).values + (test.kitch_sq == 1).values].index
test.ix[bad_index, "kitch_sq"] = np.NaN
bad_index = train[(train.full_sq > 210) & (train.life_sq / train.full_sq < 0.3)].index
train.ix[bad_index, "full_sq"] = np.NaN
bad_index = test[(test.full_sq > 150) & (test.life_sq / test.full_sq < 0.3)].index
test.ix[bad_index, "full_sq"] = np.NaN
bad_index = train[train.life_sq > 300].index
train.ix[bad_index, ["life_sq", "full_sq"]] = np.NaN
bad_index = test[test.life_sq > 200].index
test.ix[bad_index, ["life_sq", "full_sq"]] = np.NaN
train.product_type.value_counts(normalize= True)
test.product_type.value_counts(normalize= True)
bad_index = train[train.build_year < 1500].index
good_index = train[train.build_year >= 1500].index

# year #
train['year'] = train['timestamp'].dt.year
test['year'] = test['timestamp'].dt.year

train.ix[bad_index, "build_year"] = np.NaN
train['age_build'] = np.NaN
test['age_build'] = np.NaN
train.ix[good_index, "age_build"] = train['year'] - train['build_year']
good_index = test[train.build_year >= 1500].index
test.ix[good_index, "age_build"] = test['year'] - test['build_year']
bad_index = test[test.build_year < 1500].index
test.ix[bad_index, "build_year"] = np.NaN
bad_index = train[train.num_room == 0].index 
train.ix[bad_index, "num_room"] = np.NaN
bad_index = test[test.num_room == 0].index 
test.ix[bad_index, "num_room"] = np.NaN
bad_index = [10076, 11621, 17764, 19390, 24007, 26713, 29172]
train.ix[bad_index, "num_room"] = np.NaN
bad_index = [3174, 7313]
test.ix[bad_index, "num_room"] = np.NaN
bad_index = train[(train.floor == 0).values * (train.max_floor == 0).values].index
train.ix[bad_index, ["max_floor", "floor"]] = np.NaN
bad_index = train[train.floor == 0].index
train.ix[bad_index, "floor"] = np.NaN
bad_index = train[train.max_floor == 0].index
train.ix[bad_index, "max_floor"] = np.NaN
bad_index = test[test.max_floor == 0].index
test.ix[bad_index, "max_floor"] = np.NaN
bad_index = train[train.floor > train.max_floor].index
train.ix[bad_index, "max_floor"] = np.NaN
bad_index = test[test.floor > test.max_floor].index
test.ix[bad_index, "max_floor"] = np.NaN
train.floor.describe(percentiles= [0.9999])
bad_index = [23584]
train.ix[bad_index, "floor"] = np.NaN
train.material.value_counts()
test.material.value_counts()
train.state.value_counts()
bad_index = train[train.state == 33].index
train.ix[bad_index, "state"] = np.NaN
test.state.value_counts()

# brings error down a lot by removing extreme price per sqm
train.loc[train.full_sq == 0, 'full_sq'] = 50
train = train[train.price_doc/train.full_sq <= 600000]
train = train[train.price_doc/train.full_sq >= 10000]

# Add month-year
month_year = (train.timestamp.dt.month + train.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
train['month_year_cnt'] = month_year.map(month_year_cnt_map)

month_year = (test.timestamp.dt.month + test.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
test['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count
week_year = (train.timestamp.dt.weekofyear + train.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
train['week_year_cnt'] = week_year.map(week_year_cnt_map)

week_year = (test.timestamp.dt.weekofyear + test.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
test['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
train['month'] = train.timestamp.dt.month
train['dow'] = train.timestamp.dt.dayofweek

test['month'] = test.timestamp.dt.month
test['dow'] = test.timestamp.dt.dayofweek




/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:49: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [5]:
#train["extra_sq"] = train["full_sq"] - train["life_sq"]
#test["extra_sq"] = test["full_sq"] - test["life_sq"]
train['floor_from_top'] =  train['max_floor'] - train['floor']
test['floor_from_top'] = test['max_floor'] - test['floor']


train['season'] = (train['month'].astype(int) / 4).astype(int) + 1
test['season'] = (test['month'].astype(int) / 4).astype(int) + 1

train['edu_ratio'] = train['preschool_education_centers_raion'] / train['school_education_centers_raion']
test['edu_ratio'] = test['preschool_education_centers_raion'] / test['school_education_centers_raion']

train['school_ratio'] = train['children_school'] / train['school_quota'].astype(float)
test['school_ratio'] = test['children_school'] / test['school_quota'].astype(float)

train['work_ratio'] = train['work_all'] / train['full_all'].astype(float)
test['work_ratio'] = test['work_all'] / test['full_all'].astype(float)

train['male_ratio'] = train['male_f'] / train['full_all'].astype(float)
test['male_ratio'] = test['male_f'] / test['full_all'].astype(float)

train['young_ratio'] = train['young_all'] / train['full_all'].astype(float)
test['young_ratio'] = test['young_all'] / test['full_all'].astype(float)

train['ekder_ratio'] = train['ekder_all'] / train['full_all'].astype(float)
test['ekder_ratio'] = test['ekder_all'] / test['full_all'].astype(float)


train['work_male_ratio'] = train['work_male'] / train['work_all'].astype(float)
test['work_male_ratio'] = test['work_male'] / test['work_all'].astype(float)

#train['0_6_all_ratio'] = train['0_6_all'] / train['full_all'].astype(float)
#test['0_6_all_ratio'] = test['0_6_all'] / test['full_all'].astype(float)

#train['7_14_all_ratio'] = train['7_14_all'] / train['full_all'].astype(float)
#test['7_14_all_ratio'] = test['7_14_all'] / test['full_all'].astype(float)

#train['0_17_all_ratio'] = train['0_17_all'] / train['full_all'].astype(float)
#test['0_17_all_ratio'] = test['0_17_all'] / test['full_all'].astype(float)

#train['16_29_all_ratio'] = train['16_29_all'] / train['full_all'].astype(float)
#test['16_29_all_ratio'] = test['16_29_all'] / test['full_all'].astype(float)

train['pd'] = train['raion_popul'] / train['area_m'].astype(float)
test['pd'] = test['raion_popul'] / test['area_m'].astype(float)

#train['0_13_all_ratio'] = train['0_13_all'] / train['full_all']
#test['0_13_all_ratio'] = test['0_13_all'] / test['full_all']

# year #
#train['year'] = train['timestamp'].dt.year
#test['year'] = test['timestamp'].dt.year

#train['age_build'] =  train['year'] - train['build_year']
#test['age_build'] = test['year'] - test['build_year']

In [6]:
print (train['season'])

0        3
1        3
2        3
3        3
4        3
5        3
6        3
7        3
8        3
9        3
10       3
11       3
12       3
13       3
14       3
15       3
16       3
17       3
18       3
19       3
20       3
21       3
22       3
23       3
24       3
25       3
26       3
27       3
28       3
29       3
        ..
30441    2
30442    2
30443    2
30444    2
30445    2
30446    2
30447    2
30448    2
30449    2
30450    2
30451    2
30452    2
30453    2
30454    2
30455    2
30456    2
30457    2
30458    2
30459    2
30460    2
30461    2
30462    2
30463    2
30464    2
30465    2
30466    2
30467    2
30468    2
30469    2
30470    2
Name: season, Length: 30396, dtype: int64


In [7]:

# Other feature engineering
train['rel_floor'] = train['floor'] / train['max_floor'].astype(float)
train['rel_kitch_sq'] = train['kitch_sq'] / train['full_sq'].astype(float)


test['rel_floor'] = test['floor'] / test['max_floor'].astype(float)
test['rel_kitch_sq'] = test['kitch_sq'] / test['full_sq'].astype(float)

train.apartment_name=train.sub_area + train['metro_km_avto'].astype(str)
test.apartment_name=test.sub_area + train['metro_km_avto'].astype(str)

train['room_size'] = train['life_sq'] / train['num_room'].astype(float)
test['room_size'] = test['life_sq'] / test['num_room'].astype(float)

y_train_m1 = train["price_doc"]
x_train_m1 = train.drop(["id", "timestamp", "year", "price_doc"], axis=1)
#test is same for models
x_test_m1 = test.drop(["id", "timestamp", "year"], axis=1)

for c in x_train_m1.columns:
    if x_train_m1[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_train_m1[c].values)) 
        x_train_m1[c] = lbl.transform(list(x_train_m1[c].values))
        #x_train.drop(c,axis=1,inplace=True)
        
for c in x_test_m1.columns:
    if x_test_m1[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_test_m1[c].values)) 
        x_test_m1[c] = lbl.transform(list(x_test_m1[c].values))
        #x_test.drop(c,axis=1,inplace=True)  




In [76]:
from sklearn.tree import DecisionTreeRegressor  
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import cross_val_score, ShuffleSplit
from sklearn.preprocessing import Imputer  
rf=RandomForestRegressor(n_estimators = 500 , oob_score = True, n_jobs=-1,random_state = 100)
X = Imputer().fit_transform(x_train_m1)
X_test = Imputer().fit_transform(x_test_m1)
rf.fit(X,y_train_m1)#进行模型的训练


JoblibKeyError: JoblibKeyError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/anaconda/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...b/python3.6/site-packages/ipykernel/__main__.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/anaconda/lib/python3.6/runpy.py in _run_code(

In [11]:
Xtest = Imputer(missing_values='NaN', strategy='mean', axis=1).fit_transform(x_test_m1)
XX = Imputer(missing_values='NaN', strategy='mean', axis=1).fit_transform(x_train_m1)
print(XX.shape)
print(x_test_m1.shape)
print(Xtest.shape)
rfpre = rf.predict(Xtest)

(30396, 307)
(7662, 307)
(7662, 307)


In [12]:
rf_output = pd.DataFrame({'id': id_test, 'price_doc': rfpre})
rf_output.head()

,id,price_doc
0,30474,5271410.602
1,30475,8353868.466
2,30476,5279444.430
3,30477,6199531.614
4,30478,4653539.548


In [13]:
rf_output.to_csv('/Users/xiechenwei/Downloads/naive_rf_sub.csv', index=False)

In [14]:
#mse_score = cross_val_score(rf, X, y_train, cv=10, scoring='mean_squared_error')

In [15]:
#mse_score = -mse_score
#rmse_score = np.sqrt(mse_score)
#print (rmse_score)

In [16]:
#xgboost model2
xgb_params_m2 = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain_m2 = xgb.DMatrix(x_train_m1, y_train_m1)
dtest = xgb.DMatrix(x_test_m1)

#cv_output = xgb.cv(xgb_params, dtrain, num_boost_round=1000, early_stopping_rounds=20,
#    verbose_eval=50, show_stdv=False)
#cv_output[['train-rmse-mean', 'test-rmse-mean']].plot()

#num_boost_rounds = len(cv_output)






In [16]:
#CV for xgboost 1
cv_test = xgb.cv(xgb_params_m2, dtrain_m2, num_boost_round=600,nfold = 4, seed = 0, stratified = True, early_stopping_rounds=50,
               verbose_eval = 1, show_stdv=True)

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=4.
  % (min_labels, self.n_folds)), Warning)


[0]	train-rmse:8.18797e+06+8835.72	test-rmse:8.19365e+06+32841.9
[1]	train-rmse:7.83738e+06+11518	test-rmse:7.84559e+06+37600.5
[2]	train-rmse:7.51195e+06+16906.2	test-rmse:7.5267e+06+38791
[3]	train-rmse:7.19788e+06+19361	test-rmse:7.21643e+06+43766


KeyboardInterrupt: 

In [17]:
model_m2 = xgb.train(dict(xgb_params_m2, silent=0), dtrain_m2, num_boost_round=586)

#fig, ax = plt.subplots(1, 1, figsize=(8, 13))
#xgb.plot_importance(model, max_num_features=50, height=0.5, ax=ax)

y_predict_m2 = model_m2.predict(dtest)
#y_predict_m2 = np.round(y_predict * 0.99)




In [18]:
#y_predict = y_predict * 0.55 + rfpre * 0.45
gunja_output_m2 = pd.DataFrame({'id': id_test, 'price_doc': y_predict_m2})
gunja_output_m2.head()

,id,price_doc
0,30474,5269656.5
1,30475,8089211.0
2,30476,5484545.0
3,30477,5999274.5
4,30478,4748237.5


In [19]:
gunja_output_m2.to_csv('/Users/xiechenwei/Downloads/naive_xgb_model1_sub.csv', index=False)

In [82]:
#stacking model2
#add model2's output as feature

stacking_x_train = x_train_m1
stacking_x_test = x_test_m1

strain = xgb.DMatrix(x_train_m1)
s_predict = model_m2.predict(dtrain_m2)
stacking_x_train['out_put'] = s_predict
stacking_x_test['out_put'] = y_predict_m2

In [84]:
for c in stacking_x_train.columns:
    if stacking_x_train[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(stacking_x_train[c].values)) 
        stacking_x_train[c] = lbl.transform(list(stacking_x_train[c].values))
        #x_train.drop(c,axis=1,inplace=True)
        
for c in stacking_x_test.columns:
    if stacking_x_test[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(stacking_x_test[c].values)) 
        stacking_x_test[c] = lbl.transform(list(stacking_x_test[c].values))
        #x_test.drop(c,axis=1,inplace=True)  



In [85]:
#xgboost model2
xgb_params_s = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain_s = xgb.DMatrix(stacking_x_train, y_train_m1)
dtest_s = xgb.DMatrix(stacking_x_test)



In [86]:
cv_test = xgb.cv(xgb_params_s, dtrain_s, num_boost_round=600,nfold = 4, seed = 0, stratified = True, early_stopping_rounds=50,
               verbose_eval = 1, show_stdv=True)

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=4.
  % (min_labels, self.n_folds)), Warning)


[0]	train-rmse:8.16283e+06+20066.1	test-rmse:8.16603e+06+42381.5
[1]	train-rmse:7.78508e+06+23898.4	test-rmse:7.7888e+06+32598.5
[2]	train-rmse:7.42887e+06+15226.5	test-rmse:7.43468e+06+37519.2
[3]	train-rmse:7.07993e+06+14897.9	test-rmse:7.08704e+06+35710.8
[4]	train-rmse:6.75618e+06+5994.23	test-rmse:6.76322e+06+29116.8
[5]	train-rmse:6.44852e+06+13212	test-rmse:6.45692e+06+38272.7
[6]	train-rmse:6.16482e+06+35018.6	test-rmse:6.17664e+06+61090.3
[7]	train-rmse:5.8983e+06+43838.8	test-rmse:5.9126e+06+66543.1
[8]	train-rmse:5.63633e+06+37705.3	test-rmse:5.65389e+06+63067.1
[9]	train-rmse:5.39574e+06+44567.6	test-rmse:5.41655e+06+74794.8
[10]	train-rmse:5.17251e+06+53144.4	test-rmse:5.19558e+06+78995.2
[11]	train-rmse:4.94462e+06+51312.6	test-rmse:4.96838e+06+76420.1
[12]	train-rmse:4.75432e+06+51540	test-rmse:4.78083e+06+77145.7
[13]	train-rmse:4.55591e+06+52940.6	test-rmse:4.58392e+06+72761.9
[14]	train-rmse:4.36287e+06+51155.7	test-rmse:4.39193e+06+70249.7
[15]	train-rmse:4.18597e+06

In [92]:
model_s1 = xgb.train(dict(xgb_params_s, silent=0), dtrain_s, num_boost_round=100)

#fig, ax = plt.subplots(1, 1, figsize=(8, 13))
#xgb.plot_importance(model, max_num_features=50, height=0.5, ax=ax)

y_predict_s1 = model_s1.predict(dtest_s)
#y_predict_m2 = np.round(y_predict * 0.99)



In [93]:
output_s1 = pd.DataFrame({'id': id_test, 'price_doc': y_predict_s1})
output_s1.head()

,id,price_doc
0,30474,5333921.0
1,30475,8300344.0
2,30476,5543392.5
3,30477,5976511.5
4,30478,4703516.0


In [89]:
output_s1.to_csv('/Users/xiechenwei/Downloads/output_s1_sub.csv', index=False)

In [20]:
#xgboost model3

train_m3 = pd.read_csv('/Users/xiechenwei/Downloads/train.csv')
test = pd.read_csv('/Users/xiechenwei/Downloads/test.csv')
id_test = test.id

#mult = .969

y_train_m3 = train_m3["price_doc"]
x_train_m3 = train_m3.drop(["id", "timestamp", "price_doc"], axis=1)
x_test = test.drop(["id", "timestamp"], axis=1)


In [21]:
xgb_params_m3 = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain_m3 = xgb.DMatrix(x_train_m3, y_train_m3)
dtest = xgb.DMatrix(x_test)

num_boost_rounds = 483  # This was the CV output, as earlier version shows
model_m3 = xgb.train(dict(xgb_params_m3, silent=0), dtrain_m3, num_boost_round= num_boost_rounds)

y_predict_m3 = model_m3.predict(dtest)
#y_predict = y_predict * 0.7 + rfpre * 0.3
output_m3 = pd.DataFrame({'id': id_test, 'price_doc': y_predict_m3})
output_m3.head()

,id,price_doc
0,30474,5451308.0
1,30475,8493146.0
2,30476,5411929.5
3,30477,5992877.5
4,30478,5191090.5


In [22]:
alala_output_m3 = pd.DataFrame({'id': id_test, 'price_doc': y_predict_m3})
alala_output_m3.head()

,id,price_doc
0,30474,5451308.0
1,30475,8493146.0
2,30476,5411929.5
3,30477,5992877.5
4,30478,5191090.5


In [23]:
alala_output_m3.to_csv('/Users/xiechenwei/Downloads/naive_xgb_model2_sub.csv', index=False)

In [116]:
#stacking model3
#add model2's output as feature

for c in x_train_m3.columns:
    if x_train_m3[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_train_m3[c].values)) 
        x_train_m3[c] = lbl.transform(list(x_train_m1[c].values))
        #x_train.drop(c,axis=1,inplace=True)
        
for c in x_test.columns:
    if x_test[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_test[c].values)) 
        x_test[c] = lbl.transform(list(x_test[c].values))
        #x_test.drop(c,axis=1,inplace=True)  


stacking_x_train_2 = x_train_m3
stacking_x_test_2 = x_test

strain_2 = xgb.DMatrix(x_train_m3)
s2_predict = model_m2.predict(dtrain_m2)
temp = pd.DataFrame({ 'price_doc': s2_predict})
t2 = temp['price_doc']
print(t2.shape)
#sub_041 = pd.read_csv('/Users/xiechenwei/Downloads/041sub.csv')
#s2_predict = s_predict
stacking_x_train_2['out_put'] = t2
stacking_x_test_2['out_put'] = y_predict_m2

(30396,)


In [117]:





for c in stacking_x_train_2.columns:
    if stacking_x_train_2[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(stacking_x_train_2[c].values)) 
        stacking_x_train_2[c] = lbl.transform(list(stacking_x_train_2[c].values))
        #x_train.drop(c,axis=1,inplace=True)
        
for c in stacking_x_test_2.columns:
    if stacking_x_test_2[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(stacking_x_test_2[c].values)) 
        stacking_x_test_2[c] = lbl.transform(list(stacking_x_test_2[c].values))
        #x_test.drop(c,axis=1,inplace=True)  




In [122]:
#xgboost model2
xgb_params_s2 = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain_s2 = xgb.DMatrix(stacking_x_train_2, y_train_m1)
dtest_s2 = xgb.DMatrix(stacking_x_test_2)



In [ ]:
cv_test = xgb.cv(xgb_params_s2, dtrain_s2, num_boost_round=600,nfold = 4, seed = 0, stratified = True, early_stopping_rounds=50,
               verbose_eval = 1, show_stdv=True)

In [126]:
model_s2 = xgb.train(dict(xgb_params_s2, silent=0), dtrain_s2, num_boost_round=150)

#fig, ax = plt.subplots(1, 1, figsize=(8, 13))
#xgb.plot_importance(model, max_num_features=50, height=0.5, ax=ax)

y_predict_s2 = model_s1.predict(dtest_s)
#y_predict_m2 = np.round(y_predict * 0.99)




XGBoostError: b'[18:19:45] src/objective/regression_obj.cc:90: Check failed: (preds.size()) == (info.labels.size()) labels are not correctly providedpreds.size=30471, label.size=30396'

In [121]:
output_s2 = pd.DataFrame({'id': id_test, 'price_doc': y_predict_s2})
output_s2.head()

,id,price_doc
0,30474,5333921.0
1,30475,8300344.0
2,30476,5543392.5
3,30477,5976511.5
4,30478,4703516.0


In [104]:
output_s2.to_csv('/Users/xiechenwei/Downloads/output_s2_sub.csv', index=False)

In [24]:
y_pre_self_m1 = rf.predict(X)
y_pre_self_m2 = model_m2.predict(dtrain_m2)
y_pre_self_m3 = model_m3.predict(dtrain_m3)

In [25]:
a = {'x1': y_pre_self_m1, 'x2': y_pre_self_m2, 'x3': y_pre_self_m3}
alala_ensemble1_train_x = pd.DataFrame.from_dict(a, orient='index')
alala_ensemble1_train_x = alala_ensemble1_train_x.T
alala_ensemble1_train_y = y_train_m3
alala_ensemble1_train_dmatrix = xgb.DMatrix(alala_ensemble1_train_x, alala_ensemble1_train_y)

In [26]:
tt = xgb.DMatrix(x_test_m1)
tt3 = xgb.DMatrix(x_test)
test_m1 = rf.predict(Xtest)
test_m2 = model_m2.predict(tt)
test_m3 = model_m3.predict(tt3)

In [27]:
print(test_m1.shape)

(7662,)


In [61]:
b = {'x1': test_m1, 'x2': test_m1, 'x3': test_m1}
alala_ensemble1_test_x = pd.DataFrame.from_dict(b, orient='index')
alala_ensemble1_test_x = alala_ensemble1_test_x.T

In [62]:
xgb_params_ensemble1 = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0
}

test_ensemble1 = xgb.DMatrix(alala_ensemble1_test_x)

num_boost_rounds = 500  # This was the CV output, as earlier version shows
model_ensemble1 = xgb.train(dict(xgb_params_ensemble1, silent=0), alala_ensemble1_train_dmatrix, num_boost_round= num_boost_rounds)

y_predict_ensemble1 = model_ensemble1.predict(test_ensemble1)
ensemble1_output = pd.DataFrame({'id': id_test, 'price_doc': y_predict_ensemble1})
ensemble1_output.head()

,id,price_doc
0,30474,5358721.0
1,30475,8488354.0
2,30476,5263095.0
3,30477,6203592.5
4,30478,4622951.5


In [64]:
ensemble1_output.to_csv('/Users/xiechenwei/Downloads/naive_ensemble_model1_sub.csv', index=False)

In [52]:
#simple blending
simple_blending = test_m1 * 0.005 + test_m2 * 0.45 + test_m3 * 0.545


In [53]:
simple_blending_output = pd.DataFrame({'id': id_test, 'price_doc': simple_blending})
simple_blending_output.head()

,id,price_doc
0,30474,5.368665e+06
1,30475,8.310679e+06
2,30476,5.443944e+06
3,30477,5.996789e+06
4,30478,4.989119e+06


In [54]:
simple_blending_output.to_csv('/Users/xiechenwei/Downloads/simple_blending_output_sub.csv', index=False)

In [55]:
sub_041 = pd.read_csv('/Users/xiechenwei/Downloads/041sub.csv')

In [56]:
sub_041.head()

,id,price_doc
0,30474,5259326.0
1,30475,8077465.5
2,30476,5276467.5
3,30477,5797741.5
4,30478,4916051.5


In [57]:
sub_041_pre = sub_041['price_doc']
print(sub_041_pre.shape)

(7662,)


In [130]:
#simple blending 2
simple_blending_2 = simple_blending * 0.082 + sub_041_pre * 0.918

In [131]:
simple_blending_2_output = pd.DataFrame({'id': id_test, 'price_doc': simple_blending_2})
simple_blending_2_output.head()

,id,price_doc
0,30474,5.268292e+06
1,30475,8.096589e+06
2,30476,5.290201e+06
3,30477,5.814063e+06
4,30478,4.922043e+06


In [132]:
simple_blending_2_output.to_csv('/Users/xiechenwei/Downloads/simple_blending_2_output_sub.csv', index=False)

In [ ]:
cv_test2 = xgb.cv(xgb_params, dtrain, num_boost_round=500,nfold = 4, seed = 0, stratified = True, early_stopping_rounds=50,
               verbose_eval = 1, show_stdv=True)

In [ ]:
#second model

# Any results you write to the current directory are saved as output.
df_train = pd.read_csv("/Users/xiechenwei/Downloads/train.csv", parse_dates=['timestamp'])
df_test = pd.read_csv("/Users/xiechenwei/Downloads/test.csv", parse_dates=['timestamp'])
df_macro = pd.read_csv("/Users/xiechenwei/Downloads/macro.csv", parse_dates=['timestamp'])

df_train.drop(df_train[df_train["life_sq"] > 7000].index, inplace=True)

y_train = df_train['price_doc'].values  * mult + 10
id_test = df_test['id']

df_train.drop(['id', 'price_doc'], axis=1, inplace=True)
df_test.drop(['id'], axis=1, inplace=True)

num_train = len(df_train)
df_all = pd.concat([df_train, df_test])
# Next line just adds a lot of NA columns (becuase "join" only works on indexes)
# but somewhow it seems to affect the result
df_all = df_all.join(df_macro, on='timestamp', rsuffix='_macro')
print(df_all.shape)

# Add month-year
month_year = (df_all.timestamp.dt.month + df_all.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df_all['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count
week_year = (df_all.timestamp.dt.weekofyear + df_all.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df_all['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
df_all['month'] = df_all.timestamp.dt.month
df_all['dow'] = df_all.timestamp.dt.dayofweek

# Other feature engineering
df_all['rel_floor'] = df_all['floor'] / df_all['max_floor'].astype(float)
df_all['rel_kitch_sq'] = df_all['kitch_sq'] / df_all['full_sq'].astype(float)

# Remove timestamp column (may overfit the model in train)
df_all.drop(['timestamp', 'timestamp_macro'], axis=1, inplace=True)

df_all.head()

factorize = lambda t: pd.factorize(t[1])[0]

df_obj = df_all.select_dtypes(include=['object'])

X_all = np.c_[
    df_all.select_dtypes(exclude=['object']).values,
    np.array(list(map(factorize, df_obj.iteritems()))).T
]
print(X_all.shape)

X_train = X_all[:num_train]
X_test = X_all[num_train:]


# Deal with categorical values
df_numeric = df_all.select_dtypes(exclude=['object'])
df_obj = df_all.select_dtypes(include=['object']).copy()

for c in df_obj:
    df_obj[c] = pd.factorize(df_obj[c])[0]

df_values = pd.concat([df_numeric, df_obj], axis=1)


# Convert to numpy values
X_all = df_values.values
print(X_all.shape)

X_train = X_all[:num_train]
X_test = X_all[num_train:]

df_columns = df_values.columns


xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain = xgb.DMatrix(X_train, y_train, feature_names=df_columns)
dtest = xgb.DMatrix(X_test, feature_names=df_columns)


num_boost_round = 489  # From Bruno's original CV, I think
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_round)

y_pred = model.predict(dtest)

df_sub = pd.DataFrame({'id': id_test, 'price_doc': y_pred})

df_sub.head()
first_result = output.merge(df_sub, on="id", suffixes=['_louis','_bruno'])
first_result["price_doc"] = np.exp( .714*np.log(first_result.price_doc_louis) + 
                                    .286*np.log(first_result.price_doc_bruno) )  # multiplies out to .5 & .2
result = first_result.merge(gunja_output, on="id", suffixes=['_follow','_gunja'])

result["price_doc"] = np.exp( .7*np.log(result.price_doc_follow) + 
                              .3*np.log(result.price_doc_gunja) )
result.drop(["price_doc_louis","price_doc_bruno","price_doc_follow","price_doc_gunja"],axis=1,inplace=True)
result.head()
result.to_csv('/Users/xiechenwei/Downloads/sub2.csv', index=False)

In [ ]:
result.head()